In [8]:
import numpy as np
import pandas as pd
import asyncio
import httpx
from lxml import etree
from selenium import webdriver
import random
from fake_useragent import FakeUserAgent
import requests
import time
columns=["ASIN","Release date","Genre","Director","Actors","Producers","Starring","Run time","Language","Version"]


base_url="https://www.amazon.com/dp/"

productIds=np.load("data/productId.npy",allow_pickle=True)

ua=FakeUserAgent()
 



class MyReptile:
    columns=["ASIN","Release date","Genre","Director","Actors","Producers","Starring","Run time","Language","Version"]
    MAX_EXCEPTION_TIMES=3
    TIME_OUT=5
    iii=0

    #proxy format is as below
    #proxy_list = {'http://': 'http://8.134.138.108:8888',}


    def clear_all(self):
        self.data.clear()
        self.fault_asin.clear()
        self.neglect_asin.clear()
        self.proxy_list.clear()
        self.cookie_pool.clear()
        self.parse_fault_asin.clear()

    def __init__(self) -> None:
        self.data=[]
        self.fault_asin=[]
        self.neglect_asin=[]
        self.parse_fault_asin=[]
        self.proxy_list=[]
        self.cookie_pool=[]
        self.generate_proxy_pool()
        self.CONCURRENCY_NUMBER=len(self.proxy_list)
        self.SEMAPHORE=asyncio.Semaphore(self.CONCURRENCY_NUMBER*10)

    def generate_proxy_pool(self):
        self.proxy_list.clear()
        response=requests.get("http://127.0.0.1:5010/all/").json()[0:5]
        self.proxy_list=[proxy['proxy'] for proxy in response]


    def parseWhite(self,response,asin):
        tree = etree.HTML(response)
        keys=tree.xpath('//*[@id="detailBullets_feature_div"]/ul/li[*]/span/span[1]/text()')
        keys=[k.split("\u200f\n",1)[0].strip() for k in keys]    
        values=tree.xpath('//*[@id="detailBullets_feature_div"]/ul/li[*]/span/span[2]/text()')
        #get Genre info
        key_list=tree.xpath('//*[@id="productOverview_feature_div"]/div/table/tr[*]/td[1]/span/text()')
        value_list=tree.xpath('//*[@id="productOverview_feature_div"]/div/table/tr[*]/td[2]/span/text()')
    
        key_value_zip=zip(key_list,value_list)
        for item in key_value_zip:
            if "Genre"==item[0] or "Language"==item[0]:
                keys.append(item[0])
                values.append(item[1])
                
        #get version info
        version_list=tree.xpath('//*[@id="tmmSwatches"]/ul/li[*]/span/span/span/a/span[1]/text()')
        if len(version_list):
            keys.append("Version")
            values.append(','.join(version_list))

        #add asin info
        keys.append("ASIN")
        values.append(asin)

        keys_values_pair=dict(zip(keys,values))

        data_single=[]
        for key in self.columns:
            if key in keys_values_pair.keys():
                data_single.append(keys_values_pair[key])
            else:
                data_single.append(np.nan)

        self.data.append(data_single)

    def parseBlack(self,response,asin):
        tree = etree.HTML(response)
        keys=tree.xpath('//*[@id="btf-product-details"]/div[1]/dl[*]/dt/span/text()') 
        values=[]
        #多值属性：//*[@id="btf-product-details"]/div[1]/dl[x]/dd/a 
        #单值属性：//*[@id="btf-product-details"]/div[1]/dl[x]/dd
        for i,key in enumerate(keys):
            if key=="Directors":
                keys[i]='Director'
            elif key=='Audio languages':
                keys[i]="Language"
            mul_val_pattern=f'//*[@id="btf-product-details"]/div[1]/dl[{i+1}]/dd/a/text()'
            single_val_pattern=f'//*[@id="btf-product-details"]/div[1]/dl[{i+1}]/dd/text()'
            single_val_get=tree.xpath(single_val_pattern) 
            if not len(single_val_get):
                values.append(','.join(tree.xpath(mul_val_pattern)))
            else:
                values.append(single_val_get[0])

        #add asin info
        keys.append("ASIN")
        values.append(asin)
        #runtime
        runtime_val=tree.xpath('//*[@id="main"]/div[1]/div/div/div[2]/div[3]/div/div[2]/div[3]/div/span[@data-automation-id="runtime-badge"]/text()')
        if len(runtime_val):
            keys.append("Run time")
            values.append(','.join(runtime_val))
        #release time
        release_val=tree.xpath('//*[@id="main"]/div[1]/div/div/div[2]/div[3]/div/div[2]/div[3]/div/span[@data-automation-id="release-year-badge"]/text()')
        if len(release_val):
            keys.append("Release date")
            values.append(','.join(release_val))
        #genre
        genre_val=tree.xpath('//*[@id="main"]/div[1]/div/div/div[2]/div[3]/div/div[2]/div[4]/div/span[*]/a/text()')
        if len(genre_val):
            keys.append("Genre")
            values.append(','.join(genre_val))

        #version/format
        version_val=tree.xpath('//*[@id="btf-product-details"]/div[2]/div/div/a/span/strong/text()')
        if len(version_val):
            keys.append("Version")
            values.append(','.join(version_val))

        keys_values_pair=dict(zip(keys,values))
        data_single=[]
        for key in self.columns:
            if key in keys_values_pair.keys():
                data_single.append(keys_values_pair[key])
            else:
                data_single.append(np.nan)
        self.data.append(data_single)

    def whiteOrBlack(self,response)->int:
        tree = etree.HTML(response)
        if not len(tree.xpath('//*[@id="nav-search-label-id"]/text()')):
            return 2       
        type=tree.xpath('//*[@id="nav-search-label-id"]/text()')[0]
        if type=='Movies & TV':
            return 0
        elif type=="Prime Video":
            return 1
        else:
            return 2
        
    def parse(self,response,asin):
        judge=self.whiteOrBlack(response)     
        try:  
            if judge==0:
                self.parseWhite(response,asin)
            elif judge==1:
                self.parseBlack(response,asin)
            else:
                self.neglect_asin.append(asin)
        except:
            self.parse_fault_asin.append(asin)

    def get_proxy(self):
        proxy=random.choice(self.proxy_list)
        if len(self.proxy_list):
            return {'http://': "http://"+proxy}
        else:
            return{}

    def remove_proxy(self,proxy):
        try:
            self.proxy_list.remove(proxy)      
            print("proxy removed")
        except:
            print("remove proxy error")
        finally:
            if len(self.proxy_list)<self.CONCURRENCY_NUMBER:
                if len(self.proxy_list)>=10:
                    self.CONCURRENCY_NUMBER=len(self.proxy_list)
                else:
                    self.generate_proxy_pool()
            

    #pool中cookie数量大于20才可随机选择
    def get_cookie(self):
        if len(self.cookie_pool)>=20:
            return random.choice(self.cookie_pool)
        else:
            return None

    def remove_cookie(self,cookie):
        try:
            self.cookie_pool.remove(cookie)
            print("remove cookie")
        except:
            print("remove cookie error")
        

    def add_cookie(self,cookie):
        if cookie not in self.cookie_pool:
            self.cookie_pool.append(cookie)


    async def get_with_httpx(self,url,asin):
        headers = {
          'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36',
          'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,zh-TW;q=0.6',
          'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
          'accept-encoding': 'gzip, deflate, br',
          "Referer": "https://www.amazon.com/",
        }
        headers["User-Agent"]=ua.random
        exception_times=0
        while exception_times<self.MAX_EXCEPTION_TIMES:
            try:
                async with self.SEMAPHORE:
                    proxy=self.get_proxy()
                    async with httpx.AsyncClient(proxies=proxy,timeout=self.TIME_OUT) as client: 
                        rand_cookie=self.get_cookie()
                        #只给2次机会，第一次要是不行，第二次直接不使用原来的cookie
                        for i in range(2):
                            try:
                                response=await client.get(url=url,headers=headers,cookies=rand_cookie)
                                if i==0 and rand_cookie==None:
                                    self.add_cookie(response.cookies)
                                break                               
                            except:
                                if i==1:
                                    print(asin+"  "+"cookie 2 times"+" wrong")
                                    raise ""
                                else:
                                    self.remove_cookie(rand_cookie)
                                    rand_cookie=None

                        self.parse(response.text,asin)
                        self.iii+=1
                        print(self.iii)
                        #只要走过parse，所有的错误都是解析异常，和请求没有关系！
                        #print(proxy["http://"]+"  "+asin+" ok")
                        return
            except:
                exception_times+=1
                #print(asin+"exception times"+str(exception_times))
                if exception_times==3:
                    #print(asin+" proxy error: "+proxy["http://"])
                    self.remove_proxy(proxy["http://"])
        
        self.fault_asin.append(asin)
        #print("append fault asin:"+asin)

    #just for emergency:by using selenium @warning:very low efficiency
    def get_with_selenium(self,url,asin):
        browser = webdriver.Edge() # 打开浏览器
        browser.minimize_window()  # 最小化窗口
        response=browser.get(url)#访问相对应链接
           

    async def pipeline(self,asins):
        get_movie_tasks=[self.get_with_httpx(base_url+productId,productId)for productId in asins]
        await asyncio.wait(get_movie_tasks)



productIds=productIds.reshape(-1,200)

loop_times=productIds.shape[0]



# for i in range(loop_times):
#     r=MyReptile()
#     #asyncio.run(r.pipeline(productIds[i]))
#     await r.pipeline(productIds[i])
#     df=pd.DataFrame(data=r.data,columns=r.columns)
#     df_fault=pd.DataFrame(data=r.fault_asin,columns=['ASIN'])
#     df_neglect=pd.DataFrame(data=r.neglect_asin,columns=['ASIN'])
#     df_parse_fault=pd.DataFrame(data=r.parse_fault_asin,columns=['ASIN'])
#     df.to_csv(f'./ans/{i+3}.csv',index=False)
#     df_fault.to_csv(f"./ans/{i+3}fault.csv",index=False)
#     df_neglect.to_csv(f"./ans/{i+3}neglect.csv",index=False) 
#     df_parse_fault.to_csv(f"./ans/{i+3}parsefault.csv",index=False) 

#     print("epoch"+str(i+3))
#     print('----------------------')
#     print(len(r.proxy_list))
#     print(len(r.cookie_pool))
#     print('----------------------')
#     break

















ValueError: cannot reshape array of size 253059 into shape (200)

In [5]:
from lxml import etree
import numpy as np
columns=["ASIN","Release date","Genre","Director","Actors","Producers","Starring","Run time","Language","Version"]
with open('sss.html','r',encoding='utf-8') as f:
    tree = etree.HTML(f.read())
    keys=tree.xpath('//*[@id="detailBullets_feature_div"]/ul/li[*]/span/span[1]/text()')
    values=tree.xpath('//*[@id="detailBullets_feature_div"]/ul/li[*]/span/span[2]/text()')
    keys=[k.split("\u200f\n",1)[0].strip() for k in keys] 
    print(keys)
#get Genre info
    key_list=tree.xpath('//*[@id="productOverview_feature_div"]/div/table/tr[*]/td[1]/span/text()')
    value_list=tree.xpath('//*[@id="productOverview_feature_div"]/div/table/tr[*]/td[2]/span/text()')
    
    key_value_zip=zip(key_list,value_list)
    for item in key_value_zip:
        if "Genre"==item[0] or "Language"==item[0]:
            keys.append(item[0])
            values.append(item[1])
    


        
#get version info
    version_list=tree.xpath('//*[@id="tmmSwatches"]/ul/li[*]/span/span/span/a/span[1]/text()')
    if len(version_list):
            keys.append("Version")
            values.append(','.join(version_list))
    keys_values_pair=dict(zip(keys,values))

    data_single=[]
    for key in columns:
        if key in keys_values_pair.keys():
            data_single.append(keys_values_pair[key])
        else:
            data_single.append(np.nan)
    print(columns)
    print(data_single)

['Aspect Ratio', 'Is Discontinued By Manufacturer', 'MPAA rating', 'Package Dimensions', 'Director', 'Media Format', 'Run time', 'Release date', 'Actors', 'Language', 'Studio', 'ASIN', 'Number of discs']
['ASIN', 'Release date', 'Genre', 'Director', 'Actors', 'Producers', 'Starring', 'Run time', 'Language', 'Version']
['B000BMY2M4', 'December 13, 2005', nan, 'Stephen Caffrey, Reynaldo Villalobos', 'Terence Knox, Stephen Caffrey, Tony Becker, Stan Foster, Ramon Franco', nan, nan, '45 hours and 18 minutes', 'Unqualified', 'DVD']


In [9]:
import numpy as np
productIds=np.load("data/productId.npy",allow_pickle=True)
productIds[5000:5005]

array(['B00000INB0', 'B000BMY2M4', 'B000I9WVQY', 'B0019CYESM',
       'B000A8AXLS'], dtype=object)

In [3]:
import requests
response=requests.get("http://www.zdopen.com/PrivateProxy/GetIP/?api=202310301622236652&akey=57ecaf25ad4e9461&fitter=2&timespan=6&type=3").json()
proxysss=[]
for ip_port in response['data']['proxy_list']:
    proxysss.append(ip_port['ip']+":"+str(ip_port['port']))
    


proxy_list=["http://"+proxy for proxy in proxysss]